In [2]:
# General modules
import os
import openai
from dotenv import load_dotenv


# Language models
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = openai_api_key
client = openai.Client()

chat_model_name = 'gpt-4o-mini'
embed_model_name = "all-MiniLM-L6-v2"

# Paths
root_dir = r'S:\SID\Analytics\Working Files\Individual\Florian\Projects\DataScience\cluster_analysis\Data\HRC'
steam_title = 'Community'



path_input = os.path.join(root_dir, steam_title, "Transcript_pinehaven_stream.txt")
path_db_prepared = os.path.join(root_dir, steam_title, "db_prepared.json")
path_db_translated = os.path.join(root_dir, steam_title, "db_translated.json")
path_db_analysed = os.path.join(root_dir, steam_title, "db_analysed.json")
path_db_embedded = os.path.join(root_dir, steam_title, "db_embedded.json")
path_db_clustered = os.path.join(root_dir, steam_title, "db_clustered.json")
path_db_final = os.path.join(root_dir, steam_title, "db_final.json")

# Transcript Preparation

In [ ]:
# Read the transcript
with open(path_input, 'r', encoding='utf-8') as file:
    transcript = file.readlines()

In [ ]:
# concatenate every 100 lines + get rid of '\n'

transcript_joined = []
for i in range(0, len(transcript), 40):
    transcript_joined.append(' '.join([line.strip() for line in transcript[i:i+100]]))


In [ ]:
len(transcript_joined)

In [ ]:
len(transcript_joined[5])

# Analysis

In [ ]:
from helper.utils import *
from helper.prompt_templates import *

configure_api(client, chat_model_name)

all_entries = []

for i in range(0, len(transcript_joined)):
    logger.info(f"Processing text {i}")

    transcript = transcript_joined[i]

    prompt_influencer = prompt_template_influencer.format(transcript=transcript)
    response = api_settings["client"].chat.completions.create(
        model=api_settings["model"],
        messages=[
            {"role": "system", "content": "You are an expert in extracting video game topics from Youtube Transcripts."},
            {"role": "user", "content": prompt_influencer},
        ],
        response_format={"type": "json_object"},
        max_tokens=4096
    )
    response_json = json.loads(response.choices[0].message.content)

    # Dynamically handle varying keys at the root of the response
    if isinstance(response_json, dict):
        for key, value in response_json.items():
            if isinstance(value, list):  # Ensure the value is a list
                all_entries.extend(value)
            else:
                logger.warning(f"Unexpected format for key '{key}' in response {i}")
    else:
        logger.warning(f"Unexpected response structure for text {i}: {response_json}")

# save the entries
with open(path_db_prepared, "w") as output_file:
    json.dump(all_entries, output_file, indent=4)

In [ ]:
# generat unique ID

from helper.utils import *

# A unique ID is generated in the new column / key "response_ID"
data = read_json(path_db_prepared)
data_prepared = generate_ID(data)
save_to_json(data_prepared, path_db_prepared)

# Sentiment Analysis

In [ ]:
all_entries = []
data_prepared = read_json(path_db_prepared)

for i in range(0, len(data_prepared)):
    entry = data_prepared[i]
    logger.info(f"Process Sentiment for text {i}")
    try:
        prompt_sentiment = prompt_template_sentiment.format(
            review=entry["Context"], topic=entry["Topic"]
        )
        response = api_settings["client"].chat.completions.create(
            model=api_settings["model"],
            messages=[
                {"role": "system", "content": "You are an expert for sentiment analysis."},
                {"role": "user", "content": prompt_sentiment},
            ],
            max_tokens=1024,
        )
        sentiment = response.choices[0].message.content.strip()
        # rename keys
        entry["topic"] = entry["Topic"]
        entry["sentiment"] = sentiment
        entry["category"] = entry["Category"]
        entry["sentence"] = entry["Context"]
        entry.pop("Context")
        entry.pop("Category")
        entry.pop("Topic")

        all_entries.append(entry)
    except Exception as e:
        logger.error(f"Error analyzing sentiment for topic '{entry['Topic']}' (Entry ID {entry['response_ID']}): {e}")
        raise



In [ ]:
# Save the entries
with open(path_db_analysed, "w") as output_file:
    json.dump(all_entries, output_file, indent=4)

# Embedding

In [ ]:
from helper.embedding import *

embed_key = "topic"  # "topic" or "sentence"

data = read_json(path_db_analysed)
embed_model = initialize_embedding_model(embed_model_name)

def process_embedding(data, embed_key):
    for i in range(0, len(data)):
        if i % 10 == 0:
            logger.info(f"Processing entry {i}")
        entry = data[i]
        text = entry[embed_key]
        embedding = embed_text(text, embed_model)
        entry["embedding"] = embedding
    return data

data_embedded = process_embedding(data, embed_key)

# Save the embedded data
with open(path_db_embedded, "w") as output_file:
    json.dump(data_embedded, output_file, indent=4)


# Clustering

In [ ]:
from helper.cluster_analysis import *

# Adjustable parameters
dimensionality_methods = ['UMAP','PCA', 'tSNE']
hdbscan_params = {"min_cluster_size": 7, "min_samples": 2, "cluster_selection_epsilon": 0.4}

# Load data
df_total = load_embedded_data(path_db_embedded)
mat = np.array(df_total['embedding'].tolist())

# Apply HDBSCAN
df_total = apply_hdbscan(
    df_total,
    mat,
    dimensionality_methods,
    hdbscan_params=hdbscan_params,
    include_2d=True,
    include_3d=True
)

# Save results
save_df_as_json(df_total, path_db_clustered)

# Cluster Naming

In [ ]:
from helper.cluster_naming import *

# Parameters
dimensionality_methods = ["UMAP",'PCA', "tSNE"]
clustering_algorithms = ["hdbscan"]  # No KMeans here
max_centers = 10

#kmeans_clusters = [15, 20, 25, 50]  # Number of clusters for KMeans

# Load data
df_total = load_json_into_df(path_db_clustered)

# Process clusters and generate names
df_total = process_clusters(
    df_total,
    dimensionality_methods,
    clustering_algorithms,
    max_centers,
    api_settings) # insert kmeans_clusters in the function when needed


# Save results
save_data_for_streamlit(df_total, path_db_final)

# HRC Steam reviews

In [3]:
# Paths
root_dir = r'S:\SID\Analytics\Working Files\Individual\Florian\Projects\DataScience\cluster_analysis\Data\HRC'
steam_title = 'Steam'


path_input = os.path.join(root_dir, steam_title, "Transcript_pinehaven_stream.txt")
path_db_prepared = os.path.join(root_dir, steam_title, "db_prepared.json")
path_db_translated = os.path.join(root_dir, steam_title, "db_translated.json")
path_db_analysed = os.path.join(root_dir, steam_title, "db_analysed.json")
path_db_embedded = os.path.join(root_dir, steam_title, "db_embedded.json")
path_db_clustered = os.path.join(root_dir, steam_title, "db_clustered.json")
path_db_final = os.path.join(root_dir, steam_title, "db_final.json")

## Redshift query

In [4]:
# My imports
from helper.redshift_conector_standalone import *

# https://store.steampowered.com/app/1166860/Rival_Stars_Horse_Racing_Desktop_Edition/

# SQL Query Redshift
sql_query = """
SELECT *
FROM steam_review
where app_id_name = '1166860_Rival_Stars_Horse_Racing_Desktop_Edition'
"""
logger.info(f"Query Redshift with: {sql_query}")

try:
    results_json, results_df = fetch_query_results(sql_query)
    # Print the first row of the DataFrame
    logger.info("Successfully fetched query results, with shape: %s", results_df.shape)
except Exception as e:
    logger.error(f"Error fetching query results: {e}")
    raise

# Save the json
parsed_json = json.loads(results_json)

# 2) Then pretty-print with indentation
save_to_json(parsed_json, path_db_prepared)

2025-01-22 09:50:19,868 - INFO - Query Redshift with: 
SELECT *
FROM steam_review
where app_id_name = '1166860_Rival_Stars_Horse_Racing_Desktop_Edition'

2025-01-22 09:50:32,716 - INFO - Successfully fetched query results, with shape: (3235, 14)
2025-01-22 09:50:32,886 - INFO - Data successfully saved to S:\SID\Analytics\Working Files\Individual\Florian\Projects\DataScience\cluster_analysis\Data\HRC\Steam\db_prepared.json


# Translation

In [ ]:
from helper.data_analysis import translate_reviews

id_column = "recommendationid"              # The column that contains unique IDs
text_col = "review_text"                    # The column that contains the text to be translated
language_col = "language"                   # The column that contains the language tag

data_translated = translate_reviews(df=results_df,
                                    file_path=path_db_translated,
                                    id_column=id_column,
                                    text_column=text_col,
                                    language_column='language')

# Save the translated data
save_df_as_json(data_translated, path_db_translated)

# Data Analysis

In [ ]:
import os
from helper.utils import *
from helper.prompt_templates import *
from helper.data_analysis import normalize_topics_key, process_entry

# Configure API
configure_api(client, chat_model_name)

data_prepared = read_json(path_db_translated)

id_column = "recommendationid"              # The column that contains unique IDs
columns_of_interest = ["review_text"]       # The column(s) that are going to be analyzed
all_entries = []                            # List to store all processed entries
processed_ids = set()                       # Set to store IDs of processed entries

# If the analyzed file already exists, load it
if os.path.exists(path_db_analysed):
    all_entries = read_json(path_db_analysed)
    processed_ids = {entry[id_column] for entry in all_entries}  # set for O(1) membership checks

# Process all unprocessed entries
for i, entry in enumerate(data_prepared):
    current_id = entry[id_column]

    # If we've already processed this entry, skip it
    if current_id in processed_ids:
        logger.info(f"Skipping entry {i} (ID: {current_id}) - already processed.")
        continue

    # Otherwise, process and append
    process_entry(
        entry,
        id_column,
        prompt_template_topic,
        prompt_template_sentiment,
        api_settings,
        columns_of_interest
    )
    all_entries.append(entry)
    processed_ids.add(current_id)  # mark as processed

    # Save intermediate progress every 10 entries
    if (i % 10) == 0 and i != 0:
        save_to_json(all_entries, path_db_analysed)
        logger.info(f"Progress saved at index {i}.")

# Final save after the loop
save_to_json(all_entries, path_db_analysed)
logger.info("All entries processed and final results saved.")


# Embedding

In [ ]:
data = read_json(path_db_analysed)

In [ ]:
data[0]['topics']

In [ ]:
from helper.embedding import *

embed_key = "topic"  # "topic" or "sentence"

data = read_json(path_db_analysed)
embed_model = initialize_embedding_model(embed_model_name)

def process_embedding(data, embed_key):
    for i in range(0, len(data)):
        if i % 10 == 0:
            logger.info(f"Processing entry {i}")

        for d_topic in data[i]["topics"]:
            if isinstance(d_topic, dict):
                d_topic["embedding"] = embed_text(d_topic[embed_key], embed_model)
    return data

data_embedded = process_embedding(data, embed_key)

# Flatten
def flatten_data(data):
    flattened = []
    for entry in data:
        base_copy = dict(entry)
        topics = base_copy.pop("topics", [])

        for topic in topics:
            new_entry = dict(base_copy)
            new_entry.update(topic)
            flattened.append(new_entry)
    return flattened

data_flattened = flatten_data(data_embedded)


# Save the embedded data
with open(path_db_embedded, "w") as output_file:
    json.dump(data_flattened, output_file, indent=4)


# Clustering

In [ ]:
from helper.cluster_analysis import *

# Adjustable parameters
dimensionality_methods = ['UMAP','PCA', 'tSNE']
hdbscan_params = {"min_cluster_size": 15, "min_samples": 5, "cluster_selection_epsilon": 0.4}

# Load data
df_total = load_embedded_data(path_db_embedded)
mat = np.array(df_total['embedding'].tolist())

# Apply HDBSCAN
df_total = apply_hdbscan(
    df_total,
    mat,
    dimensionality_methods,
    hdbscan_params=hdbscan_params,
    include_2d=True,
    include_3d=True
)

# Save results
save_df_as_json(df_total, path_db_clustered)

# Cluster Naming

In [ ]:
from helper.cluster_naming import *

# Parameters
dimensionality_methods = ["UMAP",'PCA', "tSNE"]
clustering_algorithms = ["hdbscan"]  # No KMeans here
max_centers = 10

#kmeans_clusters = [15, 20, 25, 50]  # Number of clusters for KMeans

# Load data
df_total = load_json_into_df(path_db_clustered)

df_total = process_clusters(
    df_total,
    dimensionality_methods,
    clustering_algorithms,
    max_centers,
    api_settings) # insert kmeans_clusters in the function when needed


# Save results
save_data_for_streamlit(df_total, path_db_final)